# **QUESTION1**

In [ ]:
#Importing all the necessary libraries
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import networkx as nx
import math
nltk.download('wordnet')


In [ ]:
#IMPORT THE DATASET
df=pd.read_csv("/content/drive/MyDrive/soc-sign-bitcoinotc.csv",header=None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df

In [ ]:
df.columns

Edge Representation of data

In [ ]:
def calEdgeList(df):
  edge_list = []
  for i in range(len(df)):
    first = df.iloc[i,0]
    second = df.iloc[i,1]
    edge_list.append((first, second))
  return edge_list

In [ ]:
edge_list = calEdgeList(df)
print(edge_list)

In [ ]:
# representation of adj_matrix
columns = list(set(list(df[0])+list(df[1])))
adj_mtx = pd.DataFrame(index=columns, columns=columns)
adj_mtx = adj_mtx.fillna(0) ## intializing dataframe with zeroes instead of nan's
for i in edge_list:
    adj_mtx[i[0]][i[1]] = 1

print("ADJACENCY MATRIX", "="*120)
display(adj_mtx)

In [ ]:
# the columns given in dataset 
#['6':SOURCE, '2':TARGET,'4' RATING,'1289241911.72836': TIME]

# adj_list contains all the nodes and their adjacent nodes in the list respect to each node.
adj_list={}
for i in range(df.shape[0]):
  source=df.iloc[i,0]
  tar=df.iloc[i,1]
  if(source in adj_list):
    adj_list[source].append(tar)
  else:
    adj_list[source]=list()
    adj_list[source].append(tar)

adj_list


In [ ]:

max_ver=0
for k,v in adj_list.items():
  temp=max(v)
  max_ver=max(max_ver,temp)

Representation in the form of adjaency matrix of the graph

In [ ]:
# adj_matrix contains all the nodes in row and column and also contains the values (0/1) according to the presence of the adjacent nodes.
adj_matrix=np.zeros((max_ver+1,max_ver+1))

In [ ]:
for k,v in adj_list.items():
  for u in v:
    adj_matrix[k][u]=1

In [ ]:
adj_matrix

# Print the Number of Nodes in dataset

In [ ]:


#unique_nodes contains all the nodes
unique_nodes=set()
for k,v in adj_list.items():
  unique_nodes.add(k)
  for i in v:
    unique_nodes.add(i)
print("The total number of nodes in the dataset is: ",len(unique_nodes))

# Print the no of edges in dataset

In [ ]:
#count_edge contains count of edges of graph
count_edge=0
for k,v in adj_list.items():
  count_edge+=len(v)
print("The total number of edges in the dataset is: ",count_edge)

Average. Out-Degree

In [ ]:

#Calculating the average of out-degree of the graph
print("The Average of Outdegree of the nodes in the dataset is: ",count_edge/len(adj_list))

Average in degree

In [ ]:
#Calculating the average of in-degree of the graph
InDegree=dict()
for k,v in adj_list.items():
  for i in v:
    if i in InDegree:
      InDegree[i]+=1
    else:
      InDegree[i]=1
count_indegree=0
for k,v in InDegree.items():
  count_indegree+=v

print("The Average of Indegree of the nodes in the dataset is: ",count_indegree/len(InDegree))
     

# **NODE WITH MAXIMUM OUT DEGREE**

In [ ]:
#Finding out the node which contains maximum out-degree
max_outdegree=0
max_outnode=0
for k,v in adj_list.items():
   if max_outdegree < len(v):
    max_outdegree=max(max_outdegree,len(v))
    max_outnode=k
print("The Maximum Outdegree in the dataset is: ",max_outnode)

# **NODE WITH MAXIMUM IN -DEGREE**

In [ ]:
#Finding out the node which contains maximum in-degree
max_indegree=0
max_innode=0
for k,v in InDegree.items():
  if(max_indegree<v):
    max_indegree=max(max_indegree,v)
    max_innode=k
print("The Maximum Indegree in the dataset is: ",max_innode)

# **Density of the network**

In [ ]:
#Finding out the density of the graph network
nodes=len(unique_nodes)
print("The density of the network of the dataset is: ",count_edge/(nodes*(nodes-1)))

Plot degree distribution of the network (in case of a directed graph, plot in-degree and
out-degree separately).

In [ ]:
#Creating the dictionaries for indegree and outdegree and the respective nodes
indegree_dist=dict()
outdegree_dist=dict()
for k,v in InDegree.items():
  if(v in indegree_dist):
    indegree_dist[v]+=1
  else:
    indegree_dist[v]=1
for k,v in adj_list.items():
  temp=len(v)
  if(temp in outdegree_dist):
    outdegree_dist[temp]+=1
  else:
    outdegree_dist[temp]=1

In [ ]:
#Plot the indegree and outdegree distribution
ind_nodes = list(indegree_dist.keys())           
no_nodes = list(indegree_dist.values())        
plt.plot(ind_nodes, no_nodes, '.')
plt.xlabel("In-Degree of Nodes")
plt.ylabel("Number of Nodes")
plt.title("Degree distribution(Indegree)")
plt.show()

In [ ]:
out_nodes = list(outdegree_dist.keys())           
no_nodes = list(outdegree_dist.values())        
plt.plot(out_nodes, no_nodes, '.')
plt.xlabel("Out-Degree of Nodes")
plt.ylabel("Number of Nodes")
plt.title("Degree distribution(Outdegree)")
plt.show()

CLUSTERING COEFFICIENT OF NODE

In [ ]:
def calNeigh(nodes, adj_mtx):
  N_v, neighb = 0, []
  for j in nodes:
      if adj_mtx[i][j]:
          neighb.append(j)
  for k in neighb:
      for k2 in neighb:
          if adj_mtx[k][k2]:
              N_v += 1
  return neighb, N_v

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from tqdm import tqdm
print("\n\t\t\t\t\t   CLUSTERING CO-EFFICIENTS OF THE NODES\n")
nodes = list(set(list(df[0])+list(df[1])))
clust_coeff = {}
for i in tqdm(nodes, leave=True, position=0):
  neighb, N_v = calNeigh(nodes, adj_matrix)
  try:
      clust_coeff[i] = (N_v)/(len(neighb)*(len(neighb)-1))
  except:
      clust_coeff[i] = 0

In [ ]:
import operator
t1 = PrettyTable(['NODE','CLUSTERING-COEFFICIENT'])
for i in dict(sorted(clust_coeff.items(),key=operator.itemgetter(1),reverse=True)):
    t1.add_row([i,clust_coeff[i]])
  
print(t1)

In [ ]:
import math,operator
t1 = []
for i in dict(sorted(clust_coeff.items(),key=operator.itemgetter(1),reverse=True)):
  t1.append([i,clust_coeff[i]])
    



In [ ]:
t = pd.DataFrame(t1)
t.columns = ['Node','Coefficients']

In [ ]:
t.plot(kind = 'scatter',
        x = 'Coefficients',
        y = 'Node',
        color = 'red')
  
# set the title
plt.title('Clustering Coefficient Distribution Of The Network')
  
# show the plot
plt.show()

method 2(check)

In [ ]:
#Local cluster contains the local cluster value with respect to the node.
local_cluster=dict()
for k,v in adj_list.items():
  u=k
  count=0
  for i in v:
    for j in v:
      if adj_matrix[i][j]==1:
        count+=1
  try:
    local_cluster[k]=count/(len(v)*(len(v)-1))
  except:
    local_cluster[k]=0

In [ ]:
# file to store cluster coefficient of each node
import json
with open("lcc.json", "w") as outfile:
    json.dump(clust_coeff, outfile)

In [ ]:
#Plot local cluster coefficients and nodes.
clust_nodes = list(local_cluster.keys())           
local_coeff = list(local_cluster.values())        
plt.plot(clust_nodes, local_coeff, '.')
plt.xlabel("Nodes")
plt.ylabel("Local cluster coefficients")
plt.title("Local Clustering Coeffic")
plt.show()

In [ ]:
def calCounts(clust_coeff):
  count0, count1 = 0, 0 
  for i in clust_coeff:
    if not clust_coeff[i]:
        count0 += 1
    if clust_coeff[i] == 1:
        count1 += 1
  return count0, count1

In [ ]:
count0, count1 = calCounts(clust_coeff)

print("NODES WITH CLUSTERING COEFFICIENT 0::",count0)
print("NODES WITH CLUSTERING COEFFICIENT 1::",count1)
print("CLUSTERING COEFFICIENT OF THE NETWORK::",sum(list(clust_coeff.values()))/len(nodes))

# **QUESTION 2**

In [ ]:
#PAGE RANK OF EACH NODE
import networkx as nx
edge_list=list()
for k,v in adj_list.items():
  for u in v:
    edge_list.append((k,u))


#DG depicts the di-graph from networkx library
digraph_obj = nx.DiGraph()
digraph_obj.add_edges_from(edge_list)
#Calculating the page rank of each node from networkx library
page_rank=nx.pagerank(digraph_obj)
print(page_rank)




# **from scratch**

In [ ]:
def p_rank(G, alpha=0.85, personalization=None,max_iter=100, tol=1.0e-6, nstart=None, weight='weight',dangling=None):
    if len(G) == 0:
        return {}

    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G

    # Create a copy in (right) stochastic form
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()

    # Choose fixed starting vector if not given
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)
    else:
        # Normalized nstart vector
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())

    if personalization is None:

        # Assign uniform personalization vector if not given
        p = dict.fromkeys(W, 1.0 / N)
    else:
        missing = set(G) - set(personalization)
        if missing:
            raise NetworkXError('Personalization dictionary ''must have a value for every node.''Missing nodes %s' % missing)
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())
    if dangling is None:

        # Use personalization vector if dangling vector not specified
        dangling_weights = p
    else:
        missing = set(G) - set(dangling)
        if missing:
            raise NetworkXError('Dangling node dictionary ''must have a value for every node. ' 'Missing nodes %s' % missing)
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v/s) for k, v in dangling.items())
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
    # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        for n in x:

            # this matrix multiply looks odd because it is
            # doing a left multiply x^T=xlast^T*W
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            return x
    raise NetworkXError('pagerank: power iteration failed to converge ''in %d iterations.' % max_iter)

               

In [ ]:
G = nx.DiGraph() 
G.add_edges_from(edge_list) 
pr = p_rank(G)
pr

# **Authority and Hub score for each node**

In [ ]:
#Function to calculate the authority and hub score for each node
def calculate_hub_auth(adjacency_matrix,n):

  score_hub = list()
  for i in range(n):
    score_hub.append(1.0)
  score_authority = list()
  for i in range(n):
    score_authority.append(1.0)



  for x in range(5):
    # Calculating the authority scores of the nodes
    for j in range(n):
      temp = 0.0
      for i in range(n):
        if adjacency_matrix[i][j] == 1:
          temp += score_hub[i]
      score_authority[j] = temp
      
    # Normalizing the authority scores
    auth_sum = 0
    for i in score_authority:
      auth_sum+=i
    len_auth_score=len(score_authority)
    for i in range(len_auth_score):
      score_authority[i]=score_authority[i]/ auth_sum
    
    # Calculating the hub scores of the nodes
    for i in range(n):
      temp1 = 0.0
      for j in range(n):
        if adjacency_matrix[i][j] == 1:
          temp1 += score_authority[j]
      score_hub[i] = temp1

     # Normalizing the hub scores
    hub_sum =0
    for i in score_hub:
      hub_sum+=i
    hub_len=len(score_hub)
    for i in range(hub_len):
      score_hub[i] /= hub_sum
  temp_list=list(range(n))
  score_authority_dict=dict(zip(temp_list, score_authority))
  score_hub_dict=dict(zip(temp_list, score_hub))
  return score_authority_dict, score_hub_dict     

In [ ]:
authority_scores, hub_scores = calculate_hub_auth(adj_matrix,max_ver+1)

In [ ]:
# printing authority_scores
print(authority_scores)

In [ ]:
#printing hub score for each node
print(hub_scores)

# **Graph to show Page Rank**

In [ ]:


#Plot the Page Rank
nodes = list(page_rank.keys())           
pg_rank = list(page_rank.values())        
plt.plot(nodes, pg_rank, '.')
plt.xlabel("Nodes")
plt.ylabel("Page Rank")
plt.title("Page Rank Distribution")
plt.show()

# **Graph the Hub Score**

In [ ]:
#Graph the Hub Score
nodes = list(hub_scores.keys())           
hub_score = list(hub_scores.values())        
plt.plot(nodes,hub_score, '.')
plt.xlabel("Nodes")
plt.ylabel("Hub Scores")
plt.title("Hub Score Distribution")
plt.show()
     


# **Graph for Authority Score**

In [ ]:
#graph for the Authority Score
nodes = list(authority_scores.keys())           
auth_score = list(authority_scores.values())        
plt.plot(nodes,auth_score, '.')
plt.xlabel("Nodes")
plt.ylabel("Authority Scores")
plt.title("Authority Score Distribution")
plt.show()

# **Visualization of the Bitcoin graph**

In [ ]:
def print_Bitcoin_otc_Graph(G, title):
  plt.figure(figsize =(20, 15))
  plt.title(title)
  nx.draw_networkx(G, with_labels = True)
  return nx.hits(G,normalized = True)

In [ ]:
h,a = print_Bitcoin_otc_Graph(G, "VISUALIZATION OF THE BITCOIN NETWORK")